In [447]:
## lo básico
import pandas as pd
import numpy as np
import unicodedata
import csv
from collections import Counter


## visualización
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

## pre procesado
from sklearn.model_selection import train_test_split
#from ydata_profiling import ProfileReport

## modelado
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler











## métricas
from pandas.plotting import scatter_matrix
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve, roc_auc_score, mean_squared_error
from sklearn.metrics import classification_report, average_precision_score, ConfusionMatrixDisplay, balanced_accuracy_score,mean_absolute_error, r2_score
from sklearn.preprocessing import label_binarize

## mejora de modelos
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score

## automatización
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# 1. Carga de datos

In [448]:
url = "https://raw.githubusercontent.com/Valeriavinasl/ucm-tfm/main/data/F_E_Encuesta.csv"

df = pd.read_csv(url, sep="\t", engine="python")
df.head()

,edad,is_condicion_fisica,detalle_condicion,epoca,is_camino_realizado_prev,ruta,dias,distancia,is_calzado_adecuado,is_tr_mochila,...,hace_cuanto_Más de 2 años,temporada_Otoño,temporada_Primavera,temporada_Verano,grupo_edad_30-39,grupo_edad_40-49,grupo_edad_50-59,grupo_edad_60-70,is_volver_binaria,is_volver_talvez
0,32.0,0,No,Verano,0,Francés,5.0,100,0,0,...,1,0,0,1,1,0,0,0,0,0
1,32.0,0,No,Verano,0,Francés,5.0,100,0,0,...,1,0,0,1,1,0,0,0,0,0
2,32.0,0,No,Primavera,1,Francés,6.0,100,1,1,...,1,0,1,0,1,0,0,0,0,0
3,32.0,0,Migrañas tensionales,Primavera,0,Inglés,6.0,100,1,1,...,0,0,1,0,1,0,0,0,0,0
4,21.0,0,No,Verano,1,Francés,6.0,100,1,1,...,0,0,0,1,0,0,0,0,0,0


In [449]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 98 columns):
 #   Column                                                                               Non-Null Count  Dtype  
---  ------                                                                               --------------  -----  
 0   edad                                                                                 2000 non-null   float64
 1   is_condicion_fisica                                                                  2000 non-null   int64  
 2   detalle_condicion                                                                    2000 non-null   object 
 3   epoca                                                                                2000 non-null   object 
 4   is_camino_realizado_prev                                                             2000 non-null   int64  
 5   ruta                                                                                 2000 

# 2. Preprocesado inicial
Se realiza para evitar posibles errores más adelante. Asegura que existen las columnas interesantes, y en caso contrario, las crea a partir de otras columnas disponibles, además de comrobar que todas estén en el tipo correcto.

In [450]:
# 1 si hay cualquier tipo de si, 0 si no
if 'is_lesion' not in df.columns:
    columnas_lesion = [c for c in df.columns if c.startswith('is_lesion_')]
    if len(columnas_lesion) == 0:
        raise KeyError("No se encuentran columnas que empiecen por 'is_lesion_'.")
    df['is_lesion'] = (df[columnas_lesion].sum(axis=1) > 0).astype(int)

df['is_tr_mochila']   = df['is_tr_mochila'].astype(int)
df['is_reserva_aloj'] = df['is_reserva_aloj'].astype(int)
df['nota']            = pd.to_numeric(df['nota'], errors='coerce').astype(float)
df['intensidad_km_dia'] = pd.to_numeric(df['intensidad_km_dia'], errors='coerce').astype(float)


# 3. Targets

In [451]:
TARGET_LESION = "is_lesion"
TARGET_NOTA   = "nota"
TARGET_INTENS = "intensidad_km_dia"
TARGET_MOCH   = "is_tr_mochila"
TARGET_RESERV = "is_reserva_aloj"

drop_targets = [TARGET_LESION, TARGET_NOTA, TARGET_INTENS, TARGET_MOCH, TARGET_RESERV]

X = df.drop(columns=[c for c in drop_targets if c in df.columns], errors="ignore")
for c in [TARGET_LESION, TARGET_MOCH, TARGET_RESERV]:
    if c in df.columns:
        if df[c].dtype == bool:
            df[c] = df[c].astype(int)
        else:
            df[c] = (
                df[c]
                .replace({True:1, False:0, "Sí":1, "Si":1, "sí":1, "si":1, "Yes":1, "yes":1,
                          "No":0, "no":0, "False":0, "false":0, "0":0, "1":1})
            )

In [452]:
cv_clf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_reg = KFold(n_splits=5, shuffle=True, random_state=42)

def oof_proba_binary(estimator, X, y, cv=cv_clf):
    """
    Devuelve probabilidad out-of-fold de la clase positiva (1), alineada con el índice de X.
    """
    y = pd.Series(y).astype(int).values
    probas = cross_val_predict(estimator, X, y, cv=cv, method="predict_proba", n_jobs=-1)
    if probas.shape[1] == 2:
        return probas[:, 1]
    class_order = np.unique(y)
    pos_idx = int(np.where(class_order == 1)[0][0])
    return probas[:, pos_idx]


# 4. Splits de entramiento

In [453]:
tar = {
    "lesion": df[TARGET_LESION],
    "nota":   df[TARGET_NOTA],
    "intensidad": df[TARGET_INTENS],
    "trMochila": df[TARGET_MOCH],
    "reservaAlojamiento": df[TARGET_RESERV],
}
tipos = {
    "lesion": "clasificacion",
    "trMochila": "clasificacion",
    "reservaAlojamiento": "clasificacion",
    "nota": "regresion",
    "intensidad": "regresion",
}

In [454]:
rf_clasificacion = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced"
)
rf_regresion = RandomForestRegressor(
    n_estimators=400,
    random_state=42,
    n_jobs=-1
)

In [455]:
y_ruta  = df['ruta'].astype('category')
y_epoca = df['epoca'].astype('category')

X_ruta  = df.drop(columns=['ruta','epoca'], errors="ignore")
X_epoca = df.drop(columns=['ruta','epoca'], errors="ignore")

# solo numéricas
def make_numeric(X):
    Xn = X.select_dtypes(include=['number']).copy()
    Xn = Xn.replace({np.inf: np.nan, -np.inf: np.nan}).fillna(0)
    return Xn

X_ruta  = make_numeric(X_ruta)
X_epoca = make_numeric(X_epoca)

knn_ruta  = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5))
knn_epoca = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5))


Clasificación binaria

lesion

volver

trMochila

reservaAlojamiento
👉 Ahí ya sacás métricas como ROC-AUC y F1-score.

Regresión

nota

intensidad
👉 Ahí evaluás con R² y RMSE.

Clasificación multiclase

ruta (varias rutas posibles, codificadas one-hot)

epoca (Otoño, Primavera, Verano, etc.)
👉 Aquí es donde aplica tu saca_metricas_multiclase, porque calcula accuracy, F1 macro, y AUC macro OvR en problemas con más de 2 clases.

# 5. Clasificación binaria

In [456]:
if 'df_out' not in globals():
    df_out = df.copy()

col_prob = {
    "lesion": "prob_lesion_pred",
    "trMochila": "prob_transporte_mochila_pred",
    "reservaAlojamiento": "prob_reserva_aloj_pred",
}
for nombre, colname in col_prob.items():
    if colname not in df_out.columns:
        try:
            df_out[colname] = oof_proba_binary(rf_clasificacion, X, tar[nombre])
        except Exception as e:
            print(f"Error")

In [457]:
print("\n Métricas de Clasificación")
for nombre in ["lesion",  "trMochila", "reservaAlojamiento"]:
    y_true = tar[nombre].values
    y_prob = df_out[col_prob[nombre]].values
    y_pred = (y_prob >= 0.5).astype(int)

    try:
        roc = roc_auc_score(y_true, y_prob)
    except ValueError:
        roc = float("nan")

    f1 = f1_score(y_true, y_pred)

    print(f"{nombre:20s} | ROC-AUC: {roc:.3f} | F1-score: {f1:.3f}")


 Métricas de Clasificación
lesion               | ROC-AUC: 1.000 | F1-score: 0.955
trMochila            | ROC-AUC: 0.555 | F1-score: 0.006
reservaAlojamiento   | ROC-AUC: 0.972 | F1-score: 0.904


# 6. Regresión

In [458]:
non_num = X.select_dtypes(exclude=['number']).columns
if len(non_num):
    X = X.drop(columns=list(non_num))
X = X.replace({np.inf: np.nan, -np.inf: np.nan}).fillna(0)

tar["nota"] = pd.to_numeric(tar["nota"], errors="coerce").astype(float)
tar["intensidad"] = pd.to_numeric(tar["intensidad"], errors="coerce").astype(float)

if "nota_pred" not in df_out.columns:
    try:
        df_out["nota_pred"] = cross_val_predict(rf_regresion, X, tar["nota"], cv=cv_reg, n_jobs=-1, method="predict")
    except Exception as e:
        print(f"Error")

if "intensidad_km_dia_pred" not in df_out.columns:
    try:
        df_out["intensidad_km_dia_pred"] = cross_val_predict(rf_regresion, X, tar["intensidad"], cv=cv_reg, n_jobs=-1, method="predict")
    except Exception as e:
        print(f"Error")

In [459]:
print("\n Métricas de Regresión")
for nombre in ["nota", "intensidad"]:
    y_true = tar[nombre].values
    y_pred = df_out[col_regresion[nombre]].values

    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"{nombre:12s} | R²: {r2:.3f} | RMSE: {rmse:.3f}")



 Métricas de Regresión
nota         | R²: 0.326 | RMSE: 0.651
intensidad   | R²: 0.884 | RMSE: 2.289


# 6. Clasificación multiclase

In [460]:
def saca_metricas_multiclase(y_true, y_pred, y_proba=None, labels=None):
    print('Matriz de Confusión')
    print(confusion_matrix(y_true, y_pred))
    print('Accuracy:', accuracy_score(y_true, y_pred))
    print('Precision (macro):', precision_score(y_true, y_pred, average='macro'))
    print('Recall (macro):', recall_score(y_true, y_pred, average='macro'))
    print('F1 Score (macro):', f1_score(y_true, y_pred, average='macro'))

    if y_proba is not None and labels is not None:
        y_true_bin = label_binarize(y_true, classes=labels)
        roc_auc = roc_auc_score(y_true_bin, y_proba, average="macro", multi_class="ovr")
        print('AUC (macro, OvR):', roc_auc)

In [461]:
y_ruta  = df['ruta'].astype('category')
y_epoca = df['epoca'].astype('category')

X_ruta  = df.drop(columns=['ruta','epoca'], errors="ignore")
X_epoca = df.drop(columns=['ruta','epoca'], errors="ignore")

X_ruta  = make_numeric(X_ruta)
X_epoca = make_numeric(X_epoca)

def stratified_cv_for(y, base_splits=5):
    cnts = Counter(y)
    min_count = min(cnts.values())
    splits = min(base_splits, max(2, min_count))  # al menos 2 y como mucho la minoritaria
    return StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)

cv_ruta  = stratified_cv_for(y_ruta, base_splits=5)
cv_epoca = stratified_cv_for(y_epoca, base_splits=5)

knn_ruta  = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5))
knn_epoca = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5))

df_out["ruta_recom_vecinos"]  = cross_val_predict(knn_ruta,  X_ruta,  y_ruta,  cv=cv_ruta,  n_jobs=-1)
df_out["epoca_recom_vecinos"] = cross_val_predict(knn_epoca, X_epoca, y_epoca, cv=cv_epoca, n_jobs=-1)

In [462]:
df_out["ruta_recom_vecinos"]  = cross_val_predict(knn_ruta,  X_ruta,  y_ruta,  cv=cv_ruta,  n_jobs=-1)
df_out["epoca_recom_vecinos"] = cross_val_predict(knn_epoca, X_epoca, y_epoca, cv=cv_epoca, n_jobs=-1)

In [463]:
df_out_proba_ruta  = cross_val_predict(knn_ruta,  X_ruta,  y_ruta,  cv=cv_ruta,  n_jobs=-1, method="predict_proba")
df_out_proba_epoca = cross_val_predict(knn_epoca, X_epoca, y_epoca, cv=cv_epoca, n_jobs=-1, method="predict_proba")

In [464]:
labels_ruta  = sorted(y_ruta.unique())
labels_epoca = sorted(y_epoca.unique())

print("\n RUTA ")
saca_metricas_multiclase(y_ruta, df_out["ruta_recom_vecinos"], df_out_proba_ruta, labels_ruta)

print("\n ÉPOCA")
saca_metricas_multiclase(y_epoca, df_out["epoca_recom_vecinos"], df_out_proba_epoca, labels_epoca)


 RUTA 
Matriz de Confusión
[[  0   2   0   0   0   0]
 [  0 713  36 166   8  13]
 [  0 179  13  39   4   1]
 [  0 395  29  70   8   2]
 [  0 102  10  36   4   3]
 [  0 124   7  25   5   6]]
Accuracy: 0.403
Precision (macro): 0.19895558929758675
Recall (macro): 0.16957672778820257
F1 Score (macro): 0.15549968700867758
AUC (macro, OvR): 0.5205000212455315

 ÉPOCA
Matriz de Confusión
[[ 44  29  20 119]
 [ 40 216  13  97]
 [ 39  23 191  87]
 [ 41  34  24 983]]
Accuracy: 0.717
Precision (macro): 0.6295178633493274


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall (macro): 0.5669946456907891
F1 Score (macro): 0.5901613726124754
AUC (macro, OvR): 0.8232009662962763


In [465]:
X_num = make_numeric(X)

# 7. Importancia de variables
## 7.1 Para los targets de clasificación

In [466]:
clasificacion_targets = {
    "lesion": tar["lesion"],
    "trMochila": tar["trMochila"],
    "reservaAlojamiento": tar["reservaAlojamiento"],
}

feat_imp_clf = {}  # dict de Series

for nombre, y in clasificacion_targets.items():
    y = pd.to_numeric(y, errors="coerce").fillna(0).astype(int)
    rf = RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1)
    rf.fit(X_num, y)
    s = pd.Series(rf.feature_importances_, index=X_num.columns).sort_values(ascending=False)
    feat_imp_clf[nombre] = s
    print(f"\nTop 15 features para {nombre}")
    print(s.head(15))


Top 15 features para lesion
is_lesion_Sí, leve                                          0.356552
is_lesion_Sí, moderado                                      0.136590
is_volver_a_hacer_Tal vez                                   0.104480
is_lesion_Sí, importante (me hizo abandonar)                0.094832
edad                                                        0.023176
dias                                                        0.017048
temporada_Verano                                            0.008346
condicion_previa_Media                                      0.007773
genero_Mujer                                                0.007245
motivo_Religioso, Cultural/turístico, Personal/reflexivo    0.007112
motivo_Religioso, Deportivo, Personal/reflexivo             0.007036
hace_cuanto_Este año                                        0.006755
distancia                                                   0.006689
calzado_Zapatillas de trail running                         0.006366
is_ca

## 7.2 Para los targets de regresión

In [467]:
regresion_targets = {
    "nota": tar["nota"].astype(float),
    "intensidad": tar["intensidad"].astype(float),
}

feat_imp_reg = {}

for nombre, y in regresion_targets.items():
    rf = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
    rf.fit(X_num, y)
    s = pd.Series(rf.feature_importances_, index=X_num.columns).sort_values(ascending=False)
    feat_imp_reg[nombre] = s
    print(f"\nTop 15 features para {nombre}")
    print(s.head(15))


Top 15 features para nota
is_volver_a_hacer_Tal vez                                    0.351014
edad                                                         0.060109
dias                                                         0.045449
is_lesion_Sí, importante (me hizo abandonar)                 0.019334
is_camino_realizado_prev                                     0.014397
hace_cuanto_Este año                                         0.014290
condicion_previa_Media                                       0.013782
calzado_Zapatillas de trail running                          0.013759
calzado_Zapatillas deportivas normales                       0.013268
nivel_expereincia_previo_Moderada (10–20 km regularmente)    0.012337
genero_Mujer                                                 0.012286
temporada_Verano                                             0.012093
nivel_expereincia_previo_Básica (≤ 10 km ocasionalmente)     0.012029
dureza_Moderadas                                             0.

In [468]:
X_ruta_num  = make_numeric(X_ruta)
X_epoca_num = make_numeric(X_epoca)

## 7.3 Para la variable época

In [469]:
rf_epoca = RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1)
rf_epoca.fit(X_epoca_num, y_epoca)

importances = pd.Series(rf_epoca.feature_importances_, index=X_epoca_num.columns)
print(importances.sort_values(ascending=False).head(15))


temporada_Verano                                             0.333701
temporada_Otoño                                              0.213875
temporada_Primavera                                          0.204586
is_reserva_aloj                                              0.015917
edad                                                         0.015110
intensidad_km_dia                                            0.011841
dias                                                         0.011783
nota                                                         0.011470
condicion_previa_Media                                       0.005820
genero_Mujer                                                 0.005536
hace_cuanto_Este año                                         0.005448
nivel_expereincia_previo_Básica (≤ 10 km ocasionalmente)     0.005196
nivel_expereincia_previo_Moderada (10–20 km regularmente)    0.005004
dureza_Moderadas                                             0.004920
calzado_Zapatillas d

## 7.3 Para la variable ruta

In [470]:
rf_ruta = RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1)
rf_ruta.fit(X_ruta_num, y_ruta)

importances_ruta = pd.Series(rf_ruta.feature_importances_, index=X_ruta_num.columns)
print(importances_ruta.sort_values(ascending=False).head(15))


intensidad_km_dia                                            0.060458
dias                                                         0.057425
edad                                                         0.056883
nota                                                         0.039492
distancia                                                    0.033463
hace_cuanto_Este año                                         0.023609
condicion_previa_Media                                       0.023361
genero_Mujer                                                 0.023244
nivel_expereincia_previo_Básica (≤ 10 km ocasionalmente)     0.022378
calzado_Zapatillas de trail running                          0.019901
nivel_expereincia_previo_Moderada (10–20 km regularmente)    0.019725
dureza_Moderadas                                             0.019715
is_camino_realizado_prev                                     0.019659
calzado_Zapatillas deportivas normales                       0.019268
temporada_Verano    

# 8. Salida

In [471]:
df_out.to_csv("Salida_Modelo.csv", index=False, encoding="utf-8-sig", sep=",", quotechar='"')

In [472]:
#El anterior no lo lee bien Tableau -> .tsv
df_t = df_out.copy()

def limpiar_nombre(col):
    col = unicodedata.normalize("NFKD", col).encode("ASCII", "ignore").decode()
    for ch in [' ', ',', ';', '/', '\\', '(', ')', '[', ']', '{', '}', '"', "'"]:
        col = col.replace(ch, '_')
    col = col.replace('__','_').strip('_')
    return col

df_t.columns = [limpiar_nombre(c) for c in df_t.columns]

obj_cols = df_t.select_dtypes(include=['object']).columns
for c in obj_cols:
    df_t[c] = (df_t[c]
                  .astype(str)
                  .str.replace(r'[\r\n]+', ' ', regex=True)
                  .str.replace('\t', ' ', regex=False))

df_t.to_csv("Salida_Modelo_Tableau.tsv", sep='\t', index=False, encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL)